In [1]:
from utils import citeulike

In [2]:
user_item_matrix, item_features = citeulike()

4676 features over tag_occurence_thres (10)


In [13]:
import tensorflow as tf
#tf.enable_eager_execution()
# import tensorflow.contrib.eager as tfe
import pandas as pd
import numpy as np
import scipy.sparse as sp
import math

from tqdm import tqdm

In [8]:
uids, iids = user_item_matrix.nonzero()

In [14]:
#-------------
# HYPERPARAMS
#-------------

epochs = 50
# How many (u,i,j) triplets we sample for each batch
samples = 150
batches = math.ceil(uids.shape[0]/samples)
# batches = min(batches,30)
num_factors = 64 # Number of latent features

# Independent lambda regularization values 
# for user, items and bias.
lambda_user = 0.0000001
lambda_item = 0.0000001
lambda_bias = 0.0000001

# Our learning rate 
lr = 0.005



In [15]:
#-------------------------
# TENSORFLOW GRAPH
#-------------------------

# Set up our Tensorflow graph
graph = tf.Graph()

def init_variable(size, dim, name=None):
    '''
    Helper function to initialize a new variable with
    uniform random values.
    '''
    std = np.sqrt(2 / dim)
    return tf.Variable(tf.random_uniform([size, dim], -std, std), name=name)


def get_variable(graph, session, name):
    '''
    Helper function to get the value of a
    Tensorflow variable by name.
    '''
    v = graph.get_operation_by_name(name)
    v = v.values()[0]
    v = v.eval(session=session)
    return v

In [ ]:
def build_sparse_tensor(row,col,value,shape):
    indices = np.mat([row, col]).transpose()
    return tf.SparseTensor(indices, value, shape)

def map_one_user_item_pair(u,i,j,i_item_indices,i_item_values,j_item_indices,j_item_values):
    
    item_shape=[tf.size(i),movie_features.shape[1]]
    return (u,i,j,
            tf.SparseTensor(i_item_indices, i_item_values,item_shape),
            tf.SparseTensor(j_item_indices, j_item_values,item_shape))

def gen():
    for u,i in zip(uids, iids):
        j = np.random.randint(0, len(movies))
        i_item_row,i_item_col = movie_item_features[i].nonzero()
        i_item_values = movie_item_features[i].data
        j_item_row, j_item_col = movie_item_features[j].nonzero()
        j_item_values = movie_item_features[j].data
        i_item_indices=np.mat([i_item_row, i_item_col]).transpose()
        j_item_indices=np.mat([j_item_row, j_item_col]).transpose()
        yield (u,i,j,i_item_indices,i_item_values,j_item_indices,j_item_values)